In [15]:
""" 
dependencies:
  - python=3.8.17
  - numpy=1.24.0
  - matplotlib=3.7.1
  - pandas=2.0.2 
"""
import os
import random
import datetime
from itertools import product 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

# import random
# random.seed(42)
# np.random.seed(42)
# np.random.RandomState(42)
# os.environ['TF_DETERMINISTIC_OPS'] = '1' 

finish_sound = "afplay /Users/mehmet/Documents/vs-code/winsquare.mp3"
# play sound when finished
# os.system(finish_sound)

In [16]:
# Read data from npy file ( already preprocessed )
filename = 'original-numpy'
# filename = 'pca-numpy'
X_train = np.load(f'dataset/{filename}/X_train.npy')
X_val = np.load(f'dataset/{filename}/X_val.npy')
X_test = np.load(f'dataset/{filename}/X_test.npy')
y_train = np.load(f'dataset/{filename}/y_train.npy')
y_val = np.load(f'dataset/{filename}/y_val.npy')
y_test = np.load(f'dataset/{filename}/y_test.npy')

# # Add Bias to X
# X_train = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
# X_val = np.concatenate((np.ones((X_val.shape[0], 1)), X_val), axis=1)
# X_test = np.concatenate((np.ones((X_test.shape[0], 1)), X_test), axis=1)
# X_train.shape, X_val.shape, X_test.shape

# # Remove one hot encoding from y
# y_train = np.argmax(y_train, axis=1)
# y_val = np.argmax(y_val, axis=1)
# y_test = np.argmax(y_test, axis=1)

print(X_train.shape, y_train.shape,'\n', X_val.shape, y_val.shape,'\n', X_test.shape, y_test.shape)



(5120, 10859) (5120, 4) 
 (640, 10859) (640, 4) 
 (640, 10859) (640, 4)


In [52]:
class SVM:
    def __init__(self):
        self.weights = None
        self.bias = None


    def hinge_loss(self, scores, correct_class):
        margins = np.maximum(0, scores - scores[correct_class] + 1)
        margins[correct_class] = 0
        loss = np.sum(margins)
        return loss

    def fit(self, X, y, X_val, y_val, learning_rate=0.01, lambda_param=0.01, num_epochs=1000):
        num_samples, num_features = X.shape
        num_classes = 4

        #self.weights = np.random.rand(num_features, num_classes)
        # zero initialization
        self.weights = np.zeros((num_features, num_classes))
        self.bias = np.zeros(num_classes)

        y = np.argmax(y, axis=1)
        y_val = np.argmax(y_val, axis=1)

        for epoch in range(num_epochs):
            scores = X.dot(self.weights) + self.bias
            correct_class_mask = (np.arange(num_samples), y)

            margins = np.maximum(0, scores - scores[correct_class_mask][:, np.newaxis] + 1)
            margins[correct_class_mask] = 0

            loss = np.sum(margins)

            grad_mask = (margins > 0).astype(float)
            grad_mask[correct_class_mask] = -np.sum(grad_mask, axis=1)

            self.weights -= learning_rate * (X.T.dot(grad_mask) / num_samples + lambda_param * self.weights)
            self.bias -= learning_rate * np.sum(grad_mask, axis=0) / num_samples
            
            if epoch %300 == 0 and epoch != 0:
                learning_rate *= 1
            # For each 100 epochs print losses and accuracy
            if epoch % 100 == 0 and epoch != 0:
                # how to calculate accuracy
                predictions = self.predict(X)
                accuracy = np.mean(predictions == y)      
                val_predictions = self.predict(X_val)
                val_accuracy = np.mean(val_predictions == y_val)   
                print(f'Epoch: {epoch}, Loss: {loss}, Accuracy: {accuracy}, Val Accuracy: {val_accuracy}')

    def predict(self, X):
        scores = X.dot(self.weights) + self.bias
        predictions = np.argmax(scores, axis=1)
        return predictions


In [55]:
model = SVM()
model.fit(X_train, y_train, X_val, y_val,
        learning_rate=0.001, lambda_param=0.01, num_epochs=1000)

# Val predictions
predictions = model.predict(X_val)
print('Val. Accuracy:',np.mean(predictions == np.argmax(y_val, axis=1)))

# Test predictions
predictions = model.predict(X_test)
print('Test Accuracy:',np.mean(predictions == np.argmax(y_test, axis=1)))


Epoch: 100, Loss: 4411.720326726839, Accuracy: 0.8197265625, Val Accuracy: 0.725
Epoch: 200, Loss: 2498.1984240423826, Accuracy: 0.9052734375, Val Accuracy: 0.76875
Epoch: 300, Loss: 1335.647511217695, Accuracy: 0.9490234375, Val Accuracy: 0.8296875
Epoch: 400, Loss: 814.5437189148493, Accuracy: 0.9703125, Val Accuracy: 0.8390625
Epoch: 500, Loss: 593.1004985560048, Accuracy: 0.97421875, Val Accuracy: 0.8578125
Epoch: 600, Loss: 395.5487946886699, Accuracy: 0.9849609375, Val Accuracy: 0.8703125
Epoch: 700, Loss: 271.51468081830285, Accuracy: 0.9921875, Val Accuracy: 0.8796875
Epoch: 800, Loss: 198.9382802231686, Accuracy: 0.9947265625, Val Accuracy: 0.8859375
Epoch: 900, Loss: 189.80854333737102, Accuracy: 0.9955078125, Val Accuracy: 0.8875
Val. Accuracy: 0.89375
Test Accuracy: 0.9359375
